In [1]:
!pip install torch torchaudio numpy


In [ ]:
import torch
import torchaudio
import numpy as np
from torchaudio.models import Tacotron2
from torch import quantization
from IPython.display import Audio
import torchaudio.functional as F


model = torchaudio.models.Tacotron2.from_pretrained("tacotron2")
model.eval()

model.qconfig = torch.quantization.get_default_qconfig('fbgemm')

model_prepared = torch.quantization.prepare(model)

model_quantized = torch.quantization.convert(model_prepared)

print("Model successfully quantized.")

def preprocess_text(text):
    """
    A simple function to preprocess input text.
    - Converts the text to lowercase.
    - Other text normalization can be added.
    """
    text = text.lower()
    return text

input_text = "Hello, this is a Text-to-Speech system using quantized models."
processed_text = preprocess_text(input_text)

def text_to_speech(model, text):
    """
    Convert input text to speech.
    - Convert text to a tensor of input IDs (simulated here as ASCII values).
    - Generate the spectrogram.
    """
    input_ids = torch.tensor([ord(char) for char in text], dtype=torch.int32).unsqueeze(0)

    with torch.no_grad():
        spectrogram, _, _ = model(input_ids)

    return spectrogram

spectrogram = text_to_speech(model_quantized, processed_text)

def spectrogram_to_waveform(spectrogram):
    """
    Convert the spectrogram to waveform using Griffin-Lim vocoder.
    This is a simplified vocoder for demonstration purposes.
    """
    waveform = F.griffinlim(spectrogram)
    return waveform

waveform = spectrogram_to_waveform(spectrogram)
import torchaudio

torchaudio.save("output.wav", waveform, sample_rate=22050)

print("Audio saved as 'output.wav'.")

Audio("output.wav")
